In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

2024-01-04 12:41:24.563984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 12:41:25.443721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf_recommender = ItemKNNCFRecommender(data_train, verbose=False)
itemknncf_recommender.fit(**itemknncf_study.best_params)

[I 2024-01-04 12:41:43,606] Using an existing study with name 'ItemKNNCF' instead of creating a new one.


Similarity column 22222 (100.0%), 9561.81 column/sec. Elapsed time 2.32 sec


In [4]:
p3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3beta_recommender = RP3betaRecommender(data_train, verbose=False)
p3beta_recommender.fit(**p3beta_study.best_params)

[I 2024-01-04 12:42:08,081] Using an existing study with name 'P3Beta' instead of creating a new one.


In [5]:
study = optuna.create_study(
    study_name="ItemKNNCF + P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-04 12:44:48,697] A new study created in RDB with name: ItemKNNCF + P3Beta


In [8]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective(trial):
    alpha = trial.suggest_uniform("alpha", 0.0, 1.0)
    
    new_similarity = alpha * itemknncf_recommender.W_sparse + (1 - alpha) * p3beta_recommender.W_sparse
    
    recommender = ItemKNNCustomSimilarityRecommender(data_train, verbose=False)
    recommender.fit(new_similarity)
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=300)

/tmp/ipykernel_29671/1291327488.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 1.0)
[I 2024-01-04 13:01:27,577] Trial 41 finished with value: 0.0846003511987717 and parameters: {'alpha': 0.055985923114176}. Best is trial 10 with value: 0.08496290015265232.
[I 2024-01-04 13:01:50,720] Trial 42 finished with value: 0.08474541728895195 and parameters: {'alpha': 0.009785895309522607}. Best is trial 10 with value: 0.08496290015265232.
[I 2024-01-04 13:02:15,768] Trial 43 finished with value: 0.08492634075069547 and parameters: {'alpha': 0.03233838556854799}. Best is trial 10 with value: 0.08496290015265232.
